In [17]:
import torch
import rasterio
import importlib
import matplotlib.pyplot as plt
from simvp import SimVP
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

ImportError: module SimVP not in sys.modules

In [10]:
import torch
print(torch.__version__)
print(torch.version.cuda)


2.3.0
12.1


In [11]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Should print the name of your GPU


True
NVIDIA GeForce RTX 4060 Laptop GPU


In [12]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

import torch
x = torch.rand(10, 10, device='cuda')
y = torch.rand(10, 10, device='cuda')
z = x + y  # Test simple operation again
print(z)


tensor([[0.9847, 1.3168, 0.7707, 0.8681, 1.0586, 0.7550, 1.7364, 1.4042, 0.4854,
         1.1524],
        [0.8094, 0.6567, 0.6643, 1.1880, 1.2478, 0.1488, 1.4928, 0.8421, 1.0010,
         0.9252],
        [0.1545, 0.8506, 0.3935, 0.9808, 1.2826, 0.8525, 1.0325, 0.6767, 1.5527,
         0.8278],
        [0.7768, 0.6087, 0.5306, 0.7186, 0.7505, 1.1232, 0.7717, 0.8618, 0.6555,
         1.2053],
        [1.0064, 0.7053, 1.0961, 0.9828, 0.9536, 0.9483, 0.9726, 1.1039, 0.5345,
         1.0145],
        [1.0507, 1.4390, 1.0242, 1.1484, 0.7373, 1.3514, 0.5880, 0.6907, 1.0811,
         0.5086],
        [0.9016, 1.7818, 0.6393, 1.2980, 0.9305, 0.3179, 1.0095, 1.7220, 0.9740,
         0.6532],
        [0.5214, 0.6713, 0.4499, 1.4676, 1.2365, 1.3825, 0.8510, 0.4195, 1.6527,
         0.4609],
        [1.4087, 0.7542, 1.7315, 0.9189, 1.0927, 0.5082, 1.2749, 0.7617, 1.1106,
         0.4686],
        [1.6790, 1.8077, 1.4766, 1.1389, 0.3651, 1.2444, 0.4705, 0.5363, 0.7113,
         0.7757]], device='c

In [13]:
# root_dir = r'D:\Workspace\Projects\qpn-simvp\data\DATA_SV\Precipitation\Radar\2019\10\01\Radar_20191001000000.tif'
# root_dir2 = r'D:\Workspace\Projects\qpn-simvp\data\DATA_SV\Precipitation\Radar\2019\10\01\Radar_20191001010000.tif'
# root_dir3 = r'D:\Workspace\Projects\qpn-simvp\data\DATA_SV\Precipitation\Radar\2019\10\01\Radar_20191001020000.tif'
# img = rasterio.open(root_dir)
# data = img.read(1)  # Read the only band
# plt.figure(figsize=(10, 10))
# plt.imshow(data)
# img2 = rasterio.open(root_dir2)
# data2 = img2.read(1)  # Read the only band
# plt.figure(figsize=(10, 10))
# plt.imshow(data2)
# img3 = rasterio.open(root_dir3)
# data3 = img.read(1)  # Read the only band
# plt.figure(figsize=(10, 10))
# plt.imshow(data3)

In [14]:
class RadarTIFFDataset(Dataset):
    def __init__(self, file_paths):
        self.file_paths = file_paths
        self.transform = transforms.Compose([
            transforms.ToTensor(),  # Converts numpy array to torch tensor and scales to [0,1]
            transforms.Lambda(lambda x: x.float()),  # Ensure the tensor is float32
            transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize around the dataset mean and std dev if known
        ])

    def __len__(self):
        return len(self.file_paths) - 2  # As we need 3 images to create a single data point

    def __getitem__(self, idx):
        # Use rasterio to open and read three consecutive files
        with rasterio.open(self.file_paths[idx]) as img1:
            x1 = img1.read(1).astype(float)  # Ensure data is float before transformation
        
        with rasterio.open(self.file_paths[idx + 1]) as img2:
            x2 = img2.read(1).astype(float)
        
        with rasterio.open(self.file_paths[idx + 2]) as img3:
            y = img3.read(1).astype(float)

        # Apply transformations
        x1 = self.transform(x1)
        x2 = self.transform(x2)
        y = self.transform(y)

        # Stack x1 and x2 along a new 'time' dimension
        x = torch.stack([x1, x2], dim=0)
        return x, y

# Usage Example
file_paths = [
    r'D:\Workspace\Projects\qpn-simvp\data\DATA_SV\Precipitation\Radar\2019\10\01\Radar_20191001000000.tif',
    r'D:\Workspace\Projects\qpn-simvp\data\DATA_SV\Precipitation\Radar\2019\10\01\Radar_20191001010000.tif',
    r'D:\Workspace\Projects\qpn-simvp\data\DATA_SV\Precipitation\Radar\2019\10\01\Radar_20191001020000.tif'
]

dataset = RadarTIFFDataset(file_paths)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Example to visualize one batch of data
for x, y in dataloader:
    print("Input shape:", x.shape)  # Expect [batch_size, 2, height, width]
    print("Output shape:", y.shape)  # Expect [batch_size, 1, height, width]
    break


Input shape: torch.Size([1, 2, 1, 90, 250])
Output shape: torch.Size([1, 1, 90, 250])


In [19]:
# Assuming the necessary dimensions for hid and enc1 based on your network architecture
from simvp import Decoder
dummy_hid = torch.randn(1, 256, 60, 160)  # Adjust the size according to your Decoder's input requirements
dummy_enc1 = torch.randn(1, 256, 60, 160)  # Same here, assuming you use enc1

# Instantiate Decoder
decoder = Decoder(C_hid=256, C_out=1, N_S=5)  # Use actual values from your implementation
print("Decoder initialized successfully")
# Test the Decoder
output = decoder(dummy_hid, dummy_enc1)


Decoder initialized successfully


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 90 but got size 60 for tensor number 1 in the list.

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimVP(shape_in=(2, 3, 128, 128)).to(device)  # Adjust shape_in according to your data shape
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()
num_epochs = 10
# Training loop
for epoch in range(num_epochs):
    for x, y_true in dataloader:
        x = x.to(device)  # Send the entire batch of inputs to device
        y_true = y_true.to(device)  # Send the targets to device
        
        optimizer.zero_grad()
        y_pred = model(x)  # Forward pass
        loss = criterion(y_pred, y_true)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

RuntimeError: CUDA error: unknown error
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
